# KT Project1 1.0

## Method

### Calculate N-Gram distance of candidate pairs

In [1]:
def Ngram_distance(str1, str2, n=2):
    # add '#' in both the head and the tail
    tmp = ' ' * (n-1)
    str1 = tmp + str1 + tmp
    str2 = tmp + str2 + tmp
    set1 = set([str1[i:i+n] for i in range(len(str1)-(n-1))])
    set2 = set([str2[i:i+n] for i in range(len(str2)-(n-1))])
    setx = set1 & set2
    len1 = len(set1)
    len2 = len(set2)
    lenx = len(setx)
    num_dist = len1 + len2 - 2*lenx
    num_sim = 1 - num_dist / (len1 + len2)
    return num_dist, num_sim

### getCandidatePairs

In [2]:
def getCandidatePairs(_word, _prefixes, _suffixes):
    # candidate pairs
    candiPairs = {}
    if (len(_word) == 4):
        p = _word[:2]
        s = _word[1:]
#         print(p + " " + str(len(_prefixes[p])))
#         print(s + " " + str(len(_suffixes[s])))
        if ((p not in _prefixes) or (s not in _suffixes)):
            return candiPairs
        for i in _prefixes[p]:
            # ignore those prefix words never appear
            if (dictWithFreq[dictionary[i]] < 5): continue
            # ignore those suffix words never appear
            for j in _suffixes[s]:
                if (dictWithFreq[dictionary[j]] < 5): continue
                #the candidate pairs of the specific prefix and suffix
                pair = str(i) + ' ' + str(j)
                if (pair in candiPairs):
                    candiPairs[pair][0] += 1
                    candiPairs[pair][1].append(p + ' ' + s)
                else:
                    candiPairs[pair] = [1, [p + ' ' + s]]
    else:
        for i in range(len(_word) - 3):
            if (i + 2 >= len(_word) - i): break
            p = _word[:i + 2]
            s = _word[i + 2:]
            if ((p not in _prefixes) or (s not in _suffixes)): continue
#             print(p + " " + str(len(_prefixes[p])))
#             print(s + " " + str(len(_suffixes[s])))
            for i in _prefixes[p]:
                # ignore those prefix words never appear
                if (dictWithFreq[dictionary[i]] < 5): continue
                for j in _suffixes[s]:
                    # ignore those suffix words never appear
                    if (dictWithFreq[dictionary[j]] < 5): continue
                    #the candidate pairs of the specific prefix and suffix
                    pair = str(i) + ' ' + str(j)
                    if (pair in candiPairs):
                        candiPairs[pair][0] += 1
                        candiPairs[pair][1].append(p + ' ' + s)
                    else:
                        candiPairs[pair] = [1, [p + ' ' + s]]
    return candiPairs

### Weithting method

In [3]:
def weight(theBlend, key, w1, pre, suf):
    i1, i2 = key.split()
    s1 = dictionary[int(i1)]
    s2 = dictionary[int(i2)]
    index = 0
    w2 = 1 - w1
    # Create Inverse Coining Blend by prefix word
    for i in range(len(s1)):
        if s1[i] in "aeiou":
            if (suf[0] in "aeiou"):
                index = i
            else:
                index = i + 1
            break
    icFromP = s1[:index] + suf
    # Create Inverse Coining Blend by suffix word
    for i in range(len(s2)):
        if s2[i] in "aeiou":
            if (pre[-1] in "aeiou"):
                index = i + 1
            else:
                index = i
            break
    icFromS = pre + s2[index:]
    return w1 * Ngram_distance(theBlend, icFromP, 4)[0] + w2 * Ngram_distance(theBlend, icFromS, 4)[0]

### getDC

In [4]:
def getDC(blendWord, prefixes, suffixes, w1):
    topN = 1
    candidates = getCandidatePairs(blendWord, prefixes, suffixes)
    candidates_df = pd.DataFrame.from_dict(candidates, orient='index', columns = ['count', 'pre-suf']).reset_index().rename(columns = {'index': 'key'})
    dc = []
    for index, row in candidates_df.iterrows():
        pre, suf = row["pre-suf"][0].split()
        dc.append(weight(blendWord, row.key, w1, pre, suf))
        
    candidates_df['dc'] = dc
    pDC = (np.zeros(topN, dtype = np.int) - 1).tolist()
    sDC = (np.zeros(topN, dtype = np.int) - 1).tolist()
    pfWord = []
    sfWord = []
    pi = 0
    si = 0
    for index, row in candidates_df.sort_values(by="dc", ascending=True).iterrows():
        i1, i2 = row.key.split()
        s1 = dictionary[int(i1)]
        s2 = dictionary[int(i2)]
        if (len(pfWord) < topN and s1 not in pfWord):
            pfWord.append(s1)
            pDC[pi] = row.dc
            pi += 1;
        if (len(sfWord) < topN and s2 not in sfWord):
            sfWord.append(s2)
            sDC[si] = row.dc
            si += 1;
        if (len(pfWord) == topN and len(sfWord) == topN):
            break
#     return pfWord, sfWord, pDC, sDC
    return pDC[0]

### getTopNSufComponent

In [5]:
import pandas as pd
def getTopNSufComponent(blendWord, prefixes, suffixes, topN, w1):
    candidates = getCandidatePairs(blendWord, prefixes, suffixes)
    candidates_df = pd.DataFrame.from_dict(candidates, orient='index', columns = ['count', 'pre-suf']).reset_index().rename(columns = {'index': 'key'})
    scores = []
    for index, row in candidates_df.iterrows():
        pre, suf = row["pre-suf"][0].split()
        scores.append(weight(blendWord, row.key, w1, pre, suf))
    candidates_df['score'] = scores
    pfWord = set()
    sfWord = set()
    for x in candidates_df.sort_values(by="score", ascending=True).key:
        i1, i2 = x.split()
        s1 = dictionary[int(i1)]
        s2 = dictionary[int(i2)]
        if (len(pfWord) < topN):
            pfWord.add(s1)
        if (len(sfWord) < topN):
            sfWord.add(s2)
        if (len(pfWord) == topN and len(sfWord) == topN):
            break
    return pfWord, sfWord

## Load DATA

### Load Tweets and Preprocess

In [6]:
import os
import numpy as np
import csv
import pandas as pd
# Display progress logs on stdout
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

def readLabelData(path):
    fo = open(path, "r")
    data = fo.readlines();
    fo.close()
    res = []
    for x in data:
        x = x.rstrip('\n')
        id, tweet = x.split('\t')
        res.append([int(id), tweet])
    return res

def readTweets(path):
    fo = open(path, "r")
    data = fo.readlines();
    fo.close()
    res = []
    for x in data:
        x = x.rstrip('\n')
        res.append(x)
    return res

In [7]:
f = open("data/tweets.txt","rb")
tweets1 = []
while True:
    line = f.readline()
    if not line:
        break
    else:
        try:
            #print(line.decode('utf8'))
            tweets1.append(line.decode('utf8'))
            #为了暴露出错误，最好此处不print
        except:
            continue
#             print(str(line))
f.close()

In [8]:
tweets2 = readTweets('data/tweets2.txt')

In [9]:
df = pd.read_csv('data/tweets3.txt', 
                 encoding="utf-8",
                 header=None, sep='\t',
                quoting=csv.QUOTE_NONE)
# df = pd.DataFrame(data)
df.columns = ['id', 'tweet']
tweets3 = df.tweet.tolist()

In [10]:
tweets = tweets1 + tweets2 + tweets3
len(tweets)

426687

In [11]:
tweets_df = pd.DataFrame(tweets)
tweets_df.columns = ['tweet']
tweets_df['tweet'] = tweets_df['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
tweets_df['tweet'] = tweets_df['tweet'].str.replace('[^\w\s]','')

In [ ]:
# from nltk.corpus import stopwords
# stop = stopwords.words('english')
# tweets_df['tweet'] = tweets_df['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
# tweets_df.head()

## Load Dictionary and preprocess

In [12]:
dictionary = readTweets('data/dict.txt')
dictWithFreq = {}
for word in dictionary:
    dictWithFreq[word] = 0

In [13]:
from textblob import TextBlob
for tweet in tweets_df.tweet:
    tokens = TextBlob(tweet).words
    for token in tokens:
        if (token in dictWithFreq): dictWithFreq[token] += 1 

In [14]:
dictWithFreq_df = pd.DataFrame.from_dict(dictWithFreq, orient='index', columns = ['count']).reset_index().rename(columns = {'index': 'key'})
dictWithFreq_df.head()

,key,count
0,a,97748
1,aa,44
2,aaa,22
3,aah,17
4,aahed,0


## Load Candidate and preprocess

In [15]:
import csv
c_df = pd.read_csv("data/candidates.txt", 
                 encoding="utf-8",
                 header=None, sep='\t',
                quoting=csv.QUOTE_NONE)
c_df.columns = ['candidate']

In [16]:
candidateWithFreq = {}
for word in c_df.candidate:
    candidateWithFreq[word] = 0

In [17]:
for tweet in tweets_df.tweet:
    tokens = TextBlob(tweet).words
    for token in tokens:
        if (token in candidateWithFreq): candidateWithFreq[token] += 1 

In [18]:
candidateWithFreq_df = pd.DataFrame.from_dict(candidateWithFreq, orient='index', columns = ['count']).reset_index().rename(columns = {'index': 'key'})
candidateWithFreq_df.head()

,key,count
0,aaaaa,1
1,aaaaaaaaaa,1
2,aaaaaaaaaaa,1
3,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,1
4,aaaaaaaaaaaahhhhhhhhhhhh,1


In [19]:
candidateWithFreq_df[candidateWithFreq_df['count'] > 1].shape

(8459, 2)

## Prefixes and suffixes create

In [20]:
prefixes = {}
suffixes = {}
for word_index in range(len(dictionary)):
    # ignore word only have 1 character
    word = dictionary[word_index]
    if (len(word) >= 2):
        # add the word itself to the prefix and suffix
        if (word in prefixes):
            prefixes[word].append(word_index)
        else:
            prefixes[word] = [word_index]
        if (word in suffixes):
            suffixes[word].append(word_index)
        else:
            suffixes[word] = [word_index]
            
        # get prefix and suffix maps
        for i in range(len(word) - 3):
            pf = word[:i + 2]
            sf = word[i + 2:]
            if (pf in prefixes):
                prefixes[pf].append(word_index)
            else:
                prefixes[pf] = [word_index]
            if (sf in suffixes):
                suffixes[sf].append(word_index)
            else:
                suffixes[sf] = [word_index]

## Look into blends.txt (BAN)

In [22]:
blends_df = pd.read_csv('data/blends.txt', 
                 encoding="utf-8",
                 header=None, sep='\t',
                quoting=csv.QUOTE_NONE)
blends_df.columns = ['blend', 'pre', 'suf']

In [23]:
blends_df['pre freq'] = blends_df['pre'].apply(lambda x: dictWithFreq[x])
blends_df['suf freq'] = blends_df['suf'].apply(lambda x: dictWithFreq[x])
blends_df['dc'] = blends_df['blend'].apply(lambda x: getDC(x, prefixes, suffixes, 0.5))
blends_df.head()

,blend,pre,suf,pre freq,suf freq,dc
0,amtrak,amphibious,tractor,1,21,-1.0
1,bangel,buffy,angel,26,139,4.0
2,bash,bat,mash,66,41,0.0
3,beefaroni,beef,macaroni,184,13,7.5
4,belieber,believer,bieber,20,89,3.5


In [24]:
blends_df.sort_values(by='dc')

,blend,pre,suf,pre freq,suf freq,dc
0,amtrak,amphibious,tractor,1,21,-1.0
36,fayettenam,fayetteville,vietnam,24,50,-1.0
71,jav,japanese,av,199,38,-1.0
107,nor,not,or,18594,10693,-1.0
181,yed,your,editor,24203,152,-1.0
88,melatonin,melanin,serotonin,2,3,-1.0
146,soca,soul,calypso,434,7,-1.0
85,mar,martensite,aging,0,90,-1.0
82,malaysia,malaya,asia,1,105,-1.0
81,mahoosive,massive,huge,191,746,-1.0


In [25]:
blends_df[blends_df['dc'] >= 0].describe()

,pre freq,suf freq,dc
count,162.000000,162.000000,162.000000
mean,1966.000000,436.413580,3.000000
std,8729.247772,1814.822167,3.159822
min,0.000000,0.000000,0.000000
25%,21.250000,18.000000,0.000000
50%,162.500000,81.000000,2.500000
75%,861.750000,271.250000,6.875000
max,101460.000000,21578.000000,9.000000


## evaluation

### blend detection

In [26]:
candidateWithFreq_df['True blend'] = candidateWithFreq_df['key'].apply(lambda x: x in blends_df.blend.tolist())

In [27]:
candidateWithFreq_df.head()

,key,count,True blend
0,aaaaa,1,False
1,aaaaaaaaaa,1,False
2,aaaaaaaaaaa,1,False
3,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,1,False
4,aaaaaaaaaaaahhhhhhhhhhhh,1,False


In [48]:
dc = []
for index, row in candidateWithFreq_df.iterrows():
    isBlend = False
#     print("Processing: " + row['key'])
    ## freq > 1
    if (candidateWithFreq[row['key']] > 10):
        realdc = getDC(row['key'], prefixes, suffixes, 0.5)
        if (realdc >= 0 and realdc <= 9): isBlend = True
    dc.append(isBlend)
    

KeyboardInterrupt: 

In [ ]:
candidateWithFreq_df["Prd't Blend"] = dc

In [ ]:
TP = 0
TN = 0
FP = 0
FN = 0
for index, row in candidateWithFreq_df.iterrows():
    if (row['True blend'] == True and row["Prd't Blend"] == True):
        TP += 1
    if (row['True blend'] == False and row["Prd't Blend"] == True):
        FP += 1
    if (row['True blend'] == True and row["Prd't Blend"] == False):  
        FN += 1
    if (row['True blend'] == False and row["Prd't Blend"] == False):  
        TN += 1

In [ ]:
precision = TP / (TP + FP)
Recall = TP / (TP + FN)
accuracy = (TP + TN) / (TP + TN + FP + FN)
F1 = 2 * (precision * Recall) / (precision + Recall)
print(TP, TN, FP, FN)
print(precision, Recall, accuracy, F1)

dc 10 N 20
37 15794 739 114
0.04768041237113402 0.24503311258278146 0.9488731719012228 0.07982740021574973

dc 10 N 10
55 15228 1305 96
0.04044117647058824 0.36423841059602646 0.9160273315751618 0.07279947054930509

>2
74 13912 2621 77
0.027458256029684602 0.4900662251655629 0.8382881802924959 0.05200281096275475

>1
83 13055 3478 68
0.02330805953383881 0.5496688741721855 0.7874610405178615 0.04471982758620689

>0
96 10770 5763 55
0.01638504864311316 0.6357615894039735 0.6512826660273315 0.031946755407653904

### component detection

In [147]:
import csv
test = pd.read_csv("data/blends.txt", 
                 encoding="utf-8",
                 header=None, sep='\t',
                quoting=csv.QUOTE_NONE)
test.columns = ['Blend', 'Pre', 'Suf']
test.head()

,Blend,Pre,Suf
0,amtrak,amphibious,tractor
1,bangel,buffy,angel
2,bash,bat,mash
3,beefaroni,beef,macaroni
4,belieber,believer,bieber


In [157]:
pP = []
sP = []
for index, row in test.iterrows():
#     print("Processing: " + row['Blend'])
    pSet, sSet = getTopNSufComponent(row['Blend'], prefixes, suffixes, 1, 0.75)
    pP.append(row['Pre'] in pSet)
    sP.append(row['Suf'] in sSet)

Processing: amtrak
Processing: bangel
Processing: bash
Processing: beefaroni
Processing: belieber
Processing: bit
Processing: blaxican
Processing: bollywood
Processing: bootylicious
Processing: botox
Processing: brainiac
Processing: bromance
Processing: brunch
Processing: buppie
Processing: californication
Processing: cama
Processing: camcorder
Processing: cheeseburger
Processing: chillax
Processing: clash
Processing: clink
Processing: clois
Processing: cosplay
Processing: crackberry
Processing: cranapple
Processing: crangrape
Processing: crooklyn
Processing: crunk
Processing: daycation
Processing: dentro
Processing: dickhole
Processing: dout
Processing: dumpster
Processing: ebonics
Processing: escalator
Processing: fantabulous
Processing: fayettenam
Processing: fedex
Processing: femcee
Processing: fivehead
Processing: flunk
Processing: fooligan
Processing: frappuccino
Processing: frohawk
Processing: fruitopia
Processing: fucktard
Processing: fugly
Processing: funemployment
Processing:

In [161]:
countp = 0
counts = 0
for i in range(len(pP)):
    if (pP[i] == True):
        countp += 1
    if (sP[i] == True):
        counts += 1

print(countp / len(pP), counts / len(sP))

0.09289617486338798 0.3005464480874317


### Rank 1
0.0 0.08743169398907104 0.2896174863387978
0.5 0.08743169398907104 0.3005464480874317

### Rank 5
0.15300546448087432 0.5027322404371585